In [158]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
import numpy as np

In [ ]:
## Upload data

In [13]:
features_only=pd.read_csv('./data/features_only.csv')
features_only.head(10)

,id,question,cat_id,cat_name,answer
0,0,Что в доме Облонских?.,1368,Общество,анна б л я д ь
1,1,"Facebook блокировка, как можно узнать причину?...",1328,Интернет,чего там такого интересного? удались и не возв...
2,2,Что можно готовить на пост Марии?.,1427,Прочее кулинарное,бульбу..
3,3,Изменить голос в программе.Нужно простую запис...,1328,Интернет,"надо симитировать голос, спародировать. и прог..."
4,4,"Мне кажется, в 95 лет жизнь только начинается....",1368,Общество,Отличная идея! Обараненные рабы на раше заценят))
5,5,Как считаете с чем Вам везло или повезло в жиз...,1443,Психология,с рождением... из миллионов сперматозоидов тол...
6,6,Стоит ли брать чехол для телефона на Aliexpress?.,1508,Мобильные устройства,Да вполне на алиэкспресс чехлы стоят не дорого
7,7,"Фридрихштадтпаласт, балет - вам это о чем то г...",1469,Прочее туристическое,Видела записи. Чем-то напоминает наш Ленинград...
8,8,"Какое получится число? Если сократь дробь 3/2,...",1380,Прочие развлечения,Да
9,9,"Я слышал от Вайкуле следующее :""Нам, ЕВРОПЕЙЦА...",1369,Политика,Армянское радио отвечает:Украинцы - не европей...


In [164]:
train=pd.read_csv('./data/train.csv')
test=pd.read_csv('./data/test.csv')

In [20]:
together = pd.merge(features_only, train, on='id', how='outer')

In [21]:
together.head()

,id,question,cat_id,cat_name,answer,target
0,0,Что в доме Облонских?.,1368,Общество,анна б л я д ь,1.0
1,1,"Facebook блокировка, как можно узнать причину?...",1328,Интернет,чего там такого интересного? удались и не возв...,1.0
2,2,Что можно готовить на пост Марии?.,1427,Прочее кулинарное,бульбу..,0.0
3,3,Изменить голос в программе.Нужно простую запис...,1328,Интернет,"надо симитировать голос, спародировать. и прог...",0.0
4,4,"Мне кажется, в 95 лет жизнь только начинается....",1368,Общество,Отличная идея! Обараненные рабы на раше заценят)),2.0


In [39]:
def text_classifier(vectorizer, transformer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("transformer", transformer),
            ("classifier", classifier)]
        )

In [40]:
from sklearn.metrics  import accuracy_score

In [41]:
pip = Pipeline([("vec", CountVectorizer(ngram_range=(3,7),analyzer='char')),\
                ('tra',TfidfTransformer()),("clf", LogisticRegression(class_weight='balanced'))])

In [56]:
l=together[30000:100000]

In [57]:
pip.fit(l['answer'],l['target'])

Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(3, 7), preprocessor=None, stop_words=None,
        strip_...ty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])

In [61]:
predict = pip.predict(together['answer'][:29999])

In [64]:
accuracy_score(train['target'][:29999], predict)

0.859128637621254

In [128]:
test_by_table=features_only[1344176:]

In [76]:
test_by_table.head()

,id,question,cat_id,cat_name,answer
1344176,1344176,Почему моего сына преследуют неудачи.,1354,Прочие предсказания,Скорее всего на нем накопилось много негатива ...
1344177,1344177,Хочу взять кредит. Хочу взять в кредит 250.000...,1346,Банки и Кредиты,Маловероятно по сумме при такой зарплате.
1344178,1344178,А эту ЗАКРЫТУЮ картину узнаете и покажете? Пож...,1502,"Живопись, Графика","Аристарх Лентулов<img src=""https://otvet.imgsm..."
1344179,1344179,Тем кто был в Старбакс.В Интернете говорят что...,1414,Закуски и Салаты,поесть нормально можно. Кофе я там не беру
1344180,1344180,Ваша интуиция вас редко подводит?.,1443,Психология,Вообще то интуиция не подводит никогда проблем...


In [77]:
predict = pip.predict(features_only['answer'][1344176:])

In [93]:
ind=[]
for i in range(1344176,1374175):
    ind.append(i)

In [95]:
my_series = pd.Series(predict,index=ind)

In [129]:
test_by_table['target']=my_series

C:\Users\koval\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [155]:
answer = test_by_table[['target']]

In [135]:
answer.set_index('target', inplace=True)

In [159]:
answer.target = answer.target.astype(np.int64)

C:\Users\koval\Anaconda3\lib\site-packages\pandas\core\generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [161]:
answer.set_index('target', inplace=True)

In [163]:
answer.to_csv('./data/answer.csv')